<a href="https://colab.research.google.com/github/xiyichen/smplify-x-partial/blob/master/SMPLify_X_Partial_fitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upload the models and data files

In [ ]:
# @markdown Proceed to the [official website](https://smpl-x.is.tue.mpg.de/), register and download the zip files with SMPL-X (**models_smplx_v1_1.zip**, ~830MB), VPoser (**vposer_v1_0.zip**, ~2.5MB), and Homogenus pretrained (**homogenus_v1_0.zip**, ~42.3MB, optional) models from the **Downloads** section. 

# @markdown Run [ExPose](https://github.com/vchoutas/expose), [PIXIE](https://github.com/YadiraF/PIXIE), [PARE](https://github.com/mkocabas/PARE) optionally if you wish to use pose priors and camera initialization.

# @markdown Since uploading large zip files to Colab is relatively slow, we expect you to upload these files to Google Drive instead, link gdrive to the Colab file systems and modify variables accordingly.
%cd /content/
from google.colab import drive
drive.mount('/content/gdrive')
SMPLX_ZIP_PATH = '/content/gdrive/MyDrive/datasets/models_smplx_v1_1.zip' # @param {type:"string"}
VPOSER_ZIP_PATH = '/content/gdrive/MyDrive/datasets/vposer_v1_0.zip' # @param {type:"string"}
GENDER_CLASSIFIER_MODEL_ZIP_PATH = '/content/gdrive/MyDrive/homogenus_v1_0.zip' # @param {type:"string"}
PIXIE_RESULTS_ZIP_PATH = '/content/gdrive/MyDrive/pixie_results_croppedEHF.zip' # @param {type:"string"}
EXPOSE_RESULTS_ZIP_PATH = '/content/gdrive/MyDrive/expose_results.zip' # @param {type:"string"}
IMAGES_ZIP_PATH = '/content/gdrive/MyDrive/cropped_EHF/images.zip' # @param {type:"string"}
KEYPOINTS_ZIP_PATH = '/content/gdrive/MyDrive/blending_blend_openpose_halpe_body_openpose_face_towards_openpose_sshq_stats_standardization.zip' # @param {type:"string"}
# KEYPOINTS_ZIP_PATH = '/content/gdrive/MyDrive/blending_blend_body_openpose_face_towards_halpe_sshq_standardization.zip' # @param {type:"string"}

SMPLX_MODEL_PATH = '/content/smplx_model'
!mkdir $SMPLX_MODEL_PATH
!unzip -n $SMPLX_ZIP_PATH -d  $SMPLX_MODEL_PATH
VPOSER_MODEL_PATH = '/content/vposer'
!mkdir $VPOSER_MODEL_PATH
!unzip -n $VPOSER_ZIP_PATH -d  $VPOSER_MODEL_PATH
GENDER_CLASSIFIER_MODEL_PATH = '/content/homogenus_pretrained'
!mkdir $GENDER_CLASSIFIER_MODEL_PATH
!unzip -n $GENDER_CLASSIFIER_MODEL_ZIP_PATH -d  $GENDER_CLASSIFIER_MODEL_PATH

PIXIE_RESULTS_PATH = '/content/PIXIE_results'
!mkdir $PIXIE_RESULTS_PATH
!unzip -n $PIXIE_RESULTS_ZIP_PATH -d  $PIXIE_RESULTS_PATH

EXPOSE_RESULTS_PATH = '/content/EXPOSE_results'
!mkdir $EXPOSE_RESULTS_PATH
!unzip -n $EXPOSE_RESULTS_ZIP_PATH -d  $EXPOSE_RESULTS_PATH

DATA_PATH = '/content/data/'
!mkdir $DATA_PATH
!unzip -n $IMAGES_ZIP_PATH -d  $DATA_PATH
!unzip -n $KEYPOINTS_ZIP_PATH -d  $DATA_PATH


# Create virtual environment and install dependencies

In [ ]:
!sudo apt install python3.7
!sudo apt install python3.7-venv
!sudo apt install python3.7-dev

In [ ]:
!/usr/bin/python3.7 -m venv ~/venvs/smplify-x
!source ~/venvs/smplify-x/bin/activate

In [ ]:
!pip install -U pip
!pip install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp37-cp37m-linux_x86_64.whl
!pip install https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp37-cp37m-linux_x86_64.whl
!git clone https://github.com/xiyichen/smplx
%cd smplx
!python setup.py install
%cd ..
!pip install git+https://github.com/MPI-IS/configer.git
!wget https://github.com/nghorbani/human_body_prior/archive/refs/heads/cvpr19.zip
!pip install cvpr19.zip
!rm -fr torch-mesh-isect
!git clone https://github.com/xiyichen/torch-mesh-isect
%cd torch-mesh-isect/
!python setup.py install
%cd /content/
!git clone https://github.com/xiyichen/smplify-x-partial
%cd /content/smplify-x-partial/
!pip install -r requirements.txt
%cd smplifyx
!pip install tensorflow==1.15.2
!git clone https://github.com/xiyichen/homogenus.git
%cd homogenus
!python setup.py install
%cd /content

#Run optimization on the demo samples

In [ ]:
%cd /content/smplify-x-partial
!python smplifyx/main.py --config cfg_files/fit_smplx_combined_coco25.yaml \
    --data_folder  demo/ \
    --output_folder /content/smplifyx_results \
    --visualize=True \
    --model_folder /content/smplx_model/models \
    --vposer_ckpt /content/vposer/vposer_v1_0 \
    --use_gender_classifier True \
    --homogeneous_ckpt /content/homogenus_pretrained/homogenus_v1_0/trained_models/tf/ \
    --interpenetration True \
    --part_segm_fn smplifyx/smplx_parts_segm.pkl \
    --save_vertices True \
    --expose_results_directory demo/ExPose_results \
    --pixie_results_directory demo/PIXIE_results

#Run optimization on your uploaded samples

In [ ]:
%cd /content/smplify-x-partial
!python smplifyx/main.py --config cfg_files/fit_smplx_combined_coco25.yaml \
    --data_folder  $DATA_PATH \
    --output_folder /content/smplifyx_results \
    --visualize=True \
    --model_folder /content/smplx_model/models \
    --vposer_ckpt /content/vposer/vposer_v1_0 \
    --use_gender_classifier True \
    --homogeneous_ckpt /content/homogenus_pretrained/homogenus_v1_0/trained_models/tf/ \
    --interpenetration True \
    --part_segm_fn smplifyx/smplx_parts_segm.pkl \
    --save_vertices True \
    --expose_results_directory /content/EXPOSE_results/ \
    --pixie_results_directory /content/PIXIE_results/